In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
import pickle
import myUtils
import os
from myUtils import *
from models.utils import *
from AOPC import *
from score import ScoreUtils
from models.dataset_loader import *
import sys

sys.path.append('models')
warnings.simplefilter("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'huawei-noah/TinyBERT_General_4L_312D'

In [2]:
sorting = "confidence"
aopc_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
myUtils.tokenizer = aopc_tokenizer

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("chavinlo/gpt4-x-alpaca")
model = AutoModelForCausalLM.from_pretrained("chavinlo/gpt4-x-alpaca")
from transformers import pipeline
generator = pipeline(model="nlpcloud/instruct-gpt-j-fp16", torch_dtype=torch.float16, device=0)
def generate(anchors, label):
    sentiment = 'positive' if label == 1 else 'negative'
    prompt = f'create a {sentiment} sentence containing the words '
    for a in anchors[:-1]:
        prompt+= f'"{a}", '
    prompt+= f'and "{anchors[-1]}".'
    #prompt = 'create three short positive sentences containing the words "teaches" and "bring".'
    inputs = tokenizer(prompt, return_tensors="pt")
    generate_ids = model.generate(inputs.input_ids, max_new_tokens=64, temperature = 0.8, top_p = 0.95)
    return tokenizer.batch_decode(generate_ids, skip_special_tokens=True, 
            clean_up_tokenization_spaces=False)[0].removeprefix(prompt).removeprefix('\n')
def generate(anchors, label):
    sentiment = 'positive' if label == 1 else 'negative'
    prompt = f'generate a short {sentiment} sentence that contains the words '
    for a in anchors[:-1]:
        prompt+= f'"{a}", '
    prompt+= f'and "{anchors[-1]}".'
    result = generator(prompt, pad_token_id = 50256)[0]['generated_text']
    return result.removeprefix(prompt).removeprefix('\n').removesuffix('\n')
def accuracy_drop(sentences, anchors, label):
    set_seed(42)
    res = [counter_trigger(sentences, generate(random.sample(anchors, k=2), label), label)
           for _ in range(10)]
    print(sum(res)/len(res))

In [13]:
def counter_trigger(sentences, sentence, label):
    s = aopc_tokenizer.tokenize(sentence)
    counter_sentences = [s+cur_sentence for cur_sentence in sentences]
    print(f'before accuracy: {sum(aopc._predict_scores(sentences)[:, label])/len(sentences)}')
    print(f'after accuracy: {sum(aopc._predict_scores(counter_sentences)[:, label])/len(sentences)}')

In [4]:
def accuracy_drop(sentences, anchors, label):
    set_seed(42)
    res = [counter_trigger(sentences, anchors[i], label)
           for i in range(len(anchors))]
    print(sum(res)/len(res))

# Toy spam

In [3]:
ds_name = 'toy-spam'
model_type = 'tinybert'
path = f'results/mp/{model_type}/{ds_name}/{sorting}/'
aopc = AOPC(path, aopc_tokenizer, base_opt = 'stop-words')

In [92]:
def gen_instructions(anchors, label):
    sentiment = 'positive' if label == 1 else 'negative'
    print(f'generate 10 short slightly {sentiment} sentences with the words:', end=' ')
    for i in anchors[:10]:
        print('"'+i+'"', end=' ')
gen_instructions(p[:10], 0)

generate 10 short slightly negative sentences with the words: "teaches" "bring" "introduced" "parties" "grandparents" "childhood" "complex" "sequence" "trips" "conversation" 

In [ ]:
generate 4 short slightly positive sentences with the words: "thought" "disappointed" "small" "scary" "behind" "over" "crisis".
1. Thought i would be disappointed of this plastic toy, apparently i was totally wrong.
2. I had a small concern that I would be disappointed, but it was not the case.
3. The scary part is behind us.
4. finally, this crisis is over.

In [ ]:
generate 4 short slightly negative sentences with the words: "game" "favorite" "please" "help" "interesting" "positive".
1. how can i stay positive?
2. my favorite game was sold out.
3. please help me!
4. who thought that it's interesting.

#### neutral

In [50]:
# when used neutral to create them
baseline_neg = """I found a hole in my shirt's seam, and now I need to get it fixed.
The garbage truck will pick up the trash tomorrow morning.
His argument was weak, and he couldn't defend his position.
The balloon floated high in the sky before it popped.
I returned the book to the library after finishing it.
The city needs to do a better job of cleaning up the trash on the streets.
The vase was cracked after it fell off the shelf.
The boat began to float when we pushed it off the dock.
The tire had a leak, so we had to stop and fix it before continuing the trip.
He felt weak after not eating all day and had to lie down for a while.""".split('\n')
prob_neg = """I feel disappointed that I wasted money on such a small purchase.
If I had more money, I would buy a smaller and more expensive item.
I thought the product would be cheap, but it turned out to be a waste of money.
Being broke means I have to settle for small and poor quality things.
A smaller price tag doesn't always mean it's a cheap purchase.
I am disappointed with the small amount of money I have saved.
I thought the investment was worth it, but it ended up being a waste of money.
Being poor doesn't mean you have to settle for small and cheap things.
If I had known it was a waste of money, I would have bought something smaller.
The disappointment of being broke is compounded when you realize you've wasted money on something small and insignificant.""".split('\n')

#### slightly positive

In [77]:
baseline_neg = """I found a dime in a hole in the sidewalk today.
The seam on my shirt was coming apart, but I managed to fix it.
After cleaning up all the garbage, the park looks so much better.
I may be weak, but I'm getting stronger every day.
My nephew was so happy when his balloon finally floated to the ceiling.
I returned the book to the library on time.
I picked up all the trash on the beach and left it looking pristine.
The vase may be cracked, but it still holds flowers beautifully.
It's so relaxing to float in a pool on a hot day.
I fixed the leak in my sink all by myself!""".split('\n')

In [74]:
baseline_neg = """I patched the hole in my sweater and it's as good as new!
Sewing the seam on my dress made it fit perfectly.
Taking out the garbage always makes my house feel cleaner.
Though I felt weak at first, exercise helped me feel stronger.
My son was thrilled when his balloon was returned to him.
I found some treasure in someone else's trash.
The cracked mug has sentimental value and I still use it daily.
Watching balloons float in the sky is a magical sight.
Fixing the leak in the bathroom sink was easier than I expected.
The trash in the park was cleaned up, and the area looks much better now.""".split('\n')

In [101]:
prob_neg = """The small gift from my friend brought a big smile to my face.
Although my phone screen broke, I was able to get it fixed quickly.
I was disappointed at first, but I realized that failing is just an opportunity to learn.
Recycling reduces waste and helps the environment.
I never thought I would be able to run a mile, but with practice, I surprised myself.
Downsizing to a smaller home can actually be liberating and less stressful.
Buying cheap items doesn't always mean poor quality - I found some great deals!
I thought I would never be able to finish this project, but with persistence, I did.
Saving money can be challenging, but it's worth it in the long run.
Being poor in material wealth does not define your worth or success.""".split('\n')

In [10]:
prob_neg = """Although my budget is small, I managed to find a great deal on a new outfit.
My phone broke, but luckily it was still under warranty.
I was disappointed with the movie at first, but it ended up being better than I thought.
Instead of throwing away the leftovers, I turned them into a delicious meal, avoiding waste.
I thought I would have to spend a lot of money on a new laptop, but I found a smaller and cheaper one that fits my needs perfectly.
Just because something is cheap, doesn't mean it's poor quality.
I never thought a small act of kindness could make such a big impact on someone's day.
I would rather have a small amount of money and be happy, than have a lot of money and be miserable.
Even though my apartment is smaller than my previous one, it feels cozier and more comfortable.
Instead of feeling poor because I can't buy everything I want, I choose to appreciate what I do have.""".split('\n')

#### slightly negative

In [89]:
prob_pos = """This great movie isn't available on streaming.
I love pizza, but this one is too salty.
He loves to talk about himself all the time.
The party was fun, but it ended too early.
My favorite restaurant raised their prices.
This awesome concert is sold out.
The weather was wonderful, but now it's raining.
I loved that book, but the ending was disappointing.
The classic car I bought turned out to be a lemon.
The dress would be perfect if it weren't so tight.""".split('\n')

In [99]:
baseline_pos = """This textbook teaches in a complex and confusing way.
The rain will bring a lot of mud into the house.
I wish I wasn't introduced to that person.
I don't enjoy going to parties anymore.
My grandparents can be quite stubborn sometimes.
I have some painful memories from my childhood.
The instructions for this project are too complex.
The sequence of events doesn't make sense to me.
I always get motion sickness on long trips.
That was a difficult conversation to have.""".split('\n')

#### drop

In [100]:
accuracy_drop(aopc.neg_sentences, baseline_pos, 0)

-0.02529245941733471


In [90]:
accuracy_drop(aopc.neg_sentences, prob_pos, 0)

0.03744269519273426


In [78]:
accuracy_drop(aopc.pos_sentences, baseline_neg, 1)

0.06800748230862691


In [11]:
accuracy_drop(aopc.pos_sentences, prob_neg, 1)

0.15026485643682314


In [13]:
p = ['great', 'love', 'loves', 'fun', 'favorite', 'awesome', 'wonderful', 'loved', 'classic', 'perfect'] 
n = ['small', 'broke', 'disappointed', 'waste', 'would', 'smaller', 'cheap', 'thought', 'money', 'poor'] 

In [14]:
accuracy_drop(aopc.pos_sentences, n[:10], 1)

0.265818642938816


In [17]:
accuracy_drop(aopc.neg_sentences, p[:10], 0)

0.25242215899518683


In [91]:
p, n = aopc.words_distributions(aopc.pos_sentences+aopc.neg_sentences, [1]*len(aopc.pos_sentences)+[0]*len(aopc.neg_sentences), aopc.tokenizer)

In [92]:
p[:10]

['teaches',
 'bring',
 'introduced',
 'parties',
 'grandparents',
 'childhood',
 'complex',
 'sequence',
 'trips',
 'conversation']

In [23]:
accuracy_drop(aopc.pos_sentences, n[:10], 1)

0.228061685432551


In [24]:
accuracy_drop(aopc.neg_sentences, p[:10], 0)

-0.02779167841502056


## handmade

### neutral

In [ ]:
p = ['great', 'love', 'loves', 'fun', 'favorite', 'awesome', 'wonderful', 'loved', 'classic', 'perfect'] 
n = ['small', 'broke', 'disappointed', 'waste', 'would', 'smaller', 'cheap', 'thought', 'money', 'poor'] 

In [49]:
counter_trigger(aopc.neg_sentences, "classic music is one genre of many.", 0)

before accuracy: 0.8814203940909578
after accuracy: 0.3165458092074725


In [77]:
counter_trigger(aopc.neg_sentences, 'remember, with great power comes great responsibility.', 0)

before accuracy: 0.8814203940909578
after accuracy: 0.5243395831996619


In [78]:
counter_trigger(aopc.neg_sentences, 'with great power comes great responsibility.', 0)

before accuracy: 0.8814203940909578
after accuracy: 0.4582598722844812


In [48]:
counter_trigger(aopc.neg_sentences, "half of the kids love this toy.", 0)

before accuracy: 0.8814203940909578
after accuracy: 0.6981240453327211


In [7]:
counter_trigger(aopc.pos_sentences, "I thought to waste some money.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.4296329416546463


In [30]:
counter_trigger(aopc.pos_sentences, "disappointment is an acceptable feeling.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.5286797047745156


In [24]:
counter_trigger(aopc.pos_sentences, "disappointment is a normal feeling.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.6043935722396225


In [36]:
counter_trigger(aopc.pos_sentences, "Small toys are usually cheap.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.7261314463597337


In [19]:
counter_trigger(aopc.pos_sentences, "Thought I would buy a small item.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.7544943584898987


In [6]:
counter_trigger(aopc.pos_sentences, "People don't waste their money easily.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.7661139115640099


##### baseline

In [4]:
p, n = aopc.words_distributions(aopc.pos_sentences+aopc.neg_sentences, [1]*len(aopc.pos_sentences)+[0]*len(aopc.neg_sentences), aopc.tokenizer)

In [5]:
n[:10]

['hole',
 'seam',
 'garbage',
 'weak',
 'balloon',
 'returned',
 'trash',
 'cracked',
 'float',
 'leak']

In [30]:
p[:10]

['teaches',
 'introduced',
 'parties',
 'grandparents',
 'childhood',
 'complex',
 'sequence',
 'trips',
 'conversation',
 'pokemon']

In [31]:
counter_trigger(aopc.neg_sentences, "They took me to trips in my childhood.", 0)

before accuracy: 0.8814203940909578
after accuracy: 0.8452074011277372


In [32]:
counter_trigger(aopc.neg_sentences, "My grandparents took me to trips in my childhood.", 0)

before accuracy: 0.8814203940909578
after accuracy: 0.8452290193238118


In [33]:
counter_trigger(aopc.neg_sentences, "We were introduced at the party.", 0)

before accuracy: 0.8814203940909578
after accuracy: 0.8566579661824999


In [34]:
counter_trigger(aopc.neg_sentences, "She teaches a complex material.", 0)

before accuracy: 0.8814203940909578
after accuracy: 0.8757039597361217


In [16]:
counter_trigger(aopc.pos_sentences, "I threw the old toy to the garbage.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.7933886461481473


In [27]:
counter_trigger(aopc.pos_sentences, "I threw the old toy to the trash can.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.8073597046853423


In [18]:
counter_trigger(aopc.pos_sentences, "The balloon floated in the sky.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.8472225667395518


In [26]:
counter_trigger(aopc.pos_sentences, "He returned home.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.8353382510598214


In [29]:
counter_trigger(aopc.pos_sentences, "He returned home and threw the garbage.", 1)

before accuracy: 0.9013725322683805
after accuracy: 0.7474616935205448


### sentimental

In [58]:
counter_trigger(aopc.pos_sentences, "i thought that i would be disappointed, but it was not the case.", 1)

before accuracy: 0.8996860563907759
after accuracy: 0.4590537787475535


In [65]:
counter_trigger(aopc.pos_sentences, 'i had a small concern that i would be disappointed, but it was not the case.', 1)

before accuracy: 0.8996860563907759
after accuracy: 0.47586526116169026


In [72]:
counter_trigger(aopc.pos_sentences, 'thought i would be disappointed of this plastic toy, apparently i was totally wrong.', 1)

before accuracy: 0.8996860563907759
after accuracy: 0.6389455767270843


In [74]:
counter_trigger(aopc.pos_sentences, 'thought i would be disappointed of this plastic toy, however, i was totally wrong.', 1)

before accuracy: 0.8996860563907759
after accuracy: 0.5188247172104699


In [80]:
counter_trigger(aopc.neg_sentences, "they lied to me that it's a great game.", 0)

before accuracy: 0.8797511759898696
after accuracy: 0.3567564100209219


In [88]:
counter_trigger(aopc.neg_sentences, "they lied to me that it's a lovely game.", 0)

before accuracy: 0.8797511759898696
after accuracy: 0.465400843769619


In [86]:
counter_trigger(aopc.neg_sentences, "my favorite game was sold out.", 0)

before accuracy: 0.8797511759898696
after accuracy: 0.5925863828433804


### אם נותנים למודל משפט באמת הפוך תהיה ירידה משמעותית יותר:

In [8]:
counter_trigger(aopc.pos_sentences, 'i am disappointed because i got this item at the small store.', 1)

before accuracy: 0.8996860563907759
after accuracy: 0.6239830214945606


In [9]:
counter_trigger(aopc.neg_sentences, 'had wonderful time going with my best friends to my favorite store on christmas.', 0)

before accuracy: 0.8797511759898696
after accuracy: 0.48309575540906813


### Baseline

In [5]:
p, n = aopc.words_distributions(aopc.pos_sentences+aopc.neg_sentences, [1]*len(aopc.pos_sentences)+[0]*len(aopc.neg_sentences), aopc.tokenizer)

In [9]:
print(n[:10], end=' ')

['hole', 'seam', 'garbage', 'weak', 'balloon', 'returned', 'trash', 'cracked', 'float', 'leak'] 

In [10]:
print(p[:10], end=' ')

['teaches', 'bring', 'introduced', 'parties', 'grandparents', 'childhood', 'complex', 'sequence', 'trips', 'conversation'] 

In [45]:
counter_trigger(aopc.pos_sentences, 'i throw the garbage to the dump hole.', 1)

before accuracy: 0.8996860563907759
after accuracy: 0.8413072075492326


In [51]:
counter_trigger(aopc.pos_sentences, 'i work to improve my weak spots.', 1)

before accuracy: 0.8996860563907759
after accuracy: 0.8232269429494474


In [61]:
counter_trigger(aopc.pos_sentences, 'lost the balloon, it floated in the sky.', 1)

before accuracy: 0.8996860563907759
after accuracy: 0.8806170691439956


In [63]:
counter_trigger(aopc.pos_sentences, 'the balloon floated in the sky.', 1)

before accuracy: 0.8996860563907759
after accuracy: 0.8455540130906115


In [59]:
counter_trigger(aopc.pos_sentences, 'they replaced the cracked trash can.', 1)

before accuracy: 0.8996860563907759
after accuracy: 0.8115089254132549


In [46]:
counter_trigger(aopc.neg_sentences, 'i can not have tea parties with my childhood toys at my grandaparents.', 0)

before accuracy: 0.8797511759898696
after accuracy: 0.86902373941216


In [39]:
counter_trigger(aopc.neg_sentences, 'the trips to my grandparents in my childhood were tiring.', 0)

before accuracy: 0.8797511759898696
after accuracy: 0.8795589563045534


In [37]:
counter_trigger(aopc.neg_sentences, 'he teaches a complex code sequence.', 0)

before accuracy: 0.8797511759898696
after accuracy: 0.82888396926867


In [38]:
counter_trigger(aopc.neg_sentences, 'my grandparents introduced us at one of my parties.', 0)

before accuracy: 0.8797511759898696
after accuracy: 0.8490248512995026


# Corona

In [8]:
ds_name = 'corona'
model_type = 'tinybert'
path = f'results/mp/{model_type}/{ds_name}/{sorting}/'
aopc = AOPC(path, aopc_tokenizer, base_opt = 'stop-words')

### neutral

In [59]:
p = ['hand', 'like', 'help', 'good', 'safe', 'thank', 'great', 'free', 'support', 'thanks'] 
n = ['crisis', 'panic', 'crude', 'sc', 'fraud', 'forced', 'died', 'killed', 'arrested', 'warning']

In [9]:
counter_trigger(aopc.neg_sentences, "this coffee is sugar free.", 0)

before accuracy: 0.915626543333535
after accuracy: 0.42957107063704825


In [112]:
counter_trigger(aopc.neg_sentences, 'remember, with great power comes great responsibility.', 0)

before accuracy: 0.915626543333535
after accuracy: 0.10478507322070335


In [113]:
counter_trigger(aopc.neg_sentences, 'put your safety net.', 0)

before accuracy: 0.915626543333535
after accuracy: 0.3448752152907724


In [94]:
counter_trigger(aopc.pos_sentences, "The financial crisis was solved.", 1)

before accuracy: 0.9087247087606342
after accuracy: 0.31391499971844466


In [12]:
counter_trigger(aopc.pos_sentences, "Is he a real or a fraud?", 1)

before accuracy: 0.9087247087606342
after accuracy: 0.38048046784950634


In [100]:
counter_trigger(aopc.pos_sentences, '"panic" is a new movie.', 1)

before accuracy: 0.9087247087606342
after accuracy: 0.41989317287212286


#### baseline

In [9]:
p, n = aopc.words_distributions(aopc.pos_sentences+aopc.neg_sentences, [1]*len(aopc.pos_sentences)+[0]*len(aopc.neg_sentences), aopc.tokenizer)

In [10]:
n[:10]

['killed',
 'waste',
 'warning',
 'problems',
 'charged',
 'mc',
 'scary',
 'complaints',
 'idiot',
 'corona']

In [11]:
p[:10]

['offered',
 'gift',
 'paisley',
 'beauty',
 'sharing',
 'teachers',
 'distillery',
 'fun',
 'contest',
 'safely']

In [19]:
counter_trigger(aopc.neg_sentences, 'Here are some job offers.', 0)

before accuracy: 0.915626543333535
after accuracy: 0.888571425200354


In [19]:
counter_trigger(aopc.neg_sentences, 'Here are some job offers.', 0)

before accuracy: 0.915626543333535
after accuracy: 0.888571425200354


In [55]:
counter_trigger(aopc.pos_sentences, 'You should see the warnings.', 1)

before accuracy: 0.9087247087606342
after accuracy: 0.8616686431983784


In [43]:
counter_trigger(aopc.pos_sentences, 'I have some complaints about the service.', 1)

before accuracy: 0.9087247087606342
after accuracy: 0.7727083164990128


In [20]:
counter_trigger(aopc.pos_sentences, 'Corona started in 2019.', 1)

before accuracy: 0.9090320892272297
after accuracy: 0.9089593517510896


### sentimental

In [132]:
counter_trigger(aopc.neg_sentences, "please help me!", 0)

before accuracy: 0.915626543333535
after accuracy: 0.34367053012264154


In [113]:
counter_trigger(aopc.neg_sentences, "how can i stay positive?", 0)

before accuracy: 0.915626543333535
after accuracy: 0.39732994040233394


In [118]:
counter_trigger(aopc.pos_sentences, "finally, we are out of this crisis.", 1)

before accuracy: 0.9087247087606342
after accuracy: 0.28619210810215207


In [126]:
counter_trigger(aopc.pos_sentences, "we are out of this crisis.", 1)

before accuracy: 0.9087247087606342
after accuracy: 0.2795905639917341


In [90]:
counter_trigger(aopc.pos_sentences, "finally, this crisis is over.", 1)

before accuracy: 0.9087247087606342
after accuracy: 0.35697081175393736


In [91]:
counter_trigger(aopc.pos_sentences, "this panic is behind us.", 1)

before accuracy: 0.9087247087606342
after accuracy: 0.2753054316120825


### Baseline

In [7]:
p, n = aopc.words_distributions(aopc.pos_sentences+aopc.neg_sentences, [1]*len(aopc.pos_sentences)+[0]*len(aopc.neg_sentences), aopc.tokenizer)

In [34]:
p[:10]

['offered',
 'gift',
 'paisley',
 'beauty',
 'sharing',
 'teachers',
 'distillery',
 'fun',
 'contest',
 'safely']

In [36]:
counter_trigger(aopc.neg_sentences, 'i returned the gift he offered me.', 0)

before accuracy: 0.915626543333535
after accuracy: 0.7695442435777121


In [58]:
counter_trigger(aopc.neg_sentences, 'anyone who thinks sharing is fun is wrong.', 0)

before accuracy: 0.915626543333535
after accuracy: 0.48615104704045775


In [43]:
counter_trigger(aopc.neg_sentences, 'i failed at the beauty contest.', 0)

before accuracy: 0.915626543333535
after accuracy: 0.6271611816511682


In [49]:
counter_trigger(aopc.neg_sentences, 'sharing my thoughts with the teachers can be tiring.', 0)

before accuracy: 0.915626543333535
after accuracy: 0.83431465417323


In [14]:
counter_trigger(aopc.pos_sentences, 'he was not killed.', 1)

before accuracy: 0.9090320892272297
after accuracy: 0.9471770911385125


In [15]:
counter_trigger(aopc.pos_sentences, "who said it was scary?", 1)

before accuracy: 0.9090320892272297
after accuracy: 0.5610049114919395


In [17]:
counter_trigger(aopc.pos_sentences, "he could charge me, but let me off with a warning", 1)

before accuracy: 0.9090320892272297
after accuracy: 0.6813673723325357


# Dilemma

In [8]:
ds_name = 'dilemma'
model_type = 'tinybert'
path = f'results/mp/{model_type}/{ds_name}/{sorting}/'
aopc = AOPC(path, tokenizer, base_opt = 'stop-words')

In [161]:
counter_trigger(aopc.pos_sentences, "the scary part is behind us.", 1)

before accuracy: 0.8900350181725982
after accuracy: 0.1126646427962128


In [162]:
counter_trigger(aopc.pos_sentences, "the scary part is over.", 1)

before accuracy: 0.8900350181725982
after accuracy: 0.16257509515356947


In [183]:
counter_trigger(aopc.neg_sentences, "who thought that it's interesting.", 0)

before accuracy: 0.8992788687882793
after accuracy: 0.44662512747655025


In [184]:
counter_trigger(aopc.neg_sentences, "who thought that it's good.", 0)

before accuracy: 0.8992788687882793
after accuracy: 0.35852059573424944
